In [1]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


In [2]:
!rm -r detectron2electric/
!git clone https://github.com/DucLune/detectron2electric.git
#!unzip detectron2electric/dataset/coco-20211212.zip -d detectron2electric/dataset/
#!cp detectron2electric/JPEGImages/* detectron2electric/dataset/coco/test2014 -rf
!cp /content/detectron2electric/traincode/* ./ -rf

Cloning into 'detectron2electric'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 400 (delta 11), reused 75 (delta 11), pack-reused 325
Receiving objects: 100% (400/400), 306.86 MiB | 32.97 MiB/s, done.
Resolving deltas: 100% (188/188), done.
Checking out files: 100% (405/405), done.


In [3]:
# import some common libraries
import random
import cv2
import json
import os
import numpy as np
import argparse
import torch
import detectron2
# import some common detectron2 utilities
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import pascal_voc,register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset,PascalVOCDetectionEvaluator
from detectron2.utils.file_io import PathManager
from detectron2.utils.logger import setup_logger
from predictor import VisualizationDemo

import myPascal

In [4]:
# Some basic setup:
# Setup detectron2 logger
setup_logger()

# 创建解析
parser = argparse.ArgumentParser(description="detectron2 demo",formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# 添加参数
parser.add_argument('--train_url', type=str, default="model/glue_hole/",help='the path model saved')
parser.add_argument('--data_url', type=str,default="detectron2electric/dataset/", help='the training data')
parser.add_argument('--device', type=str, default='cuda',
                    help='the training device')
parser.add_argument('--dataset', type=str, default='coco-20211212',
                    help='the dataset dirname')
# 解析参数
args, unkown = parser.parse_known_args()


In [5]:
# 定义训练集和测试集
#class_names = ("glue","injection_hole","pin_glue","pin_inclined_side","pin_inclined_top")
#class_names = ("Component", "Injection_hole")
dirname = os.path.join(args.data_url, args.dataset)
#pascal_voc.register_pascal_voc(    "mydataset_train", dirname, split="train", year=2021, class_names=class_names)
#pascal_voc.register_pascal_voc(    "mydataset_test", dirname, split="test", year=2021, class_names=class_names)
register_coco_instances("mydataset_train", {}, os.path.join(dirname,"annotations","instances_train2014.json"), os.path.join(dirname,"train2014"))
register_coco_instances("mydataset_val", {}, os.path.join(dirname,"annotations","instances_val2014.json"), os.path.join(dirname,"val2014"))

In [6]:
# 定义模型并训练
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#cfg.merge_from_file(model_zoo.get_config_file("PascalVOC-Detection/faster_rcnn_R_50_C4.yaml"))
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = args.data_url+"/model_final_b1acc2.pkl"
cfg.MODEL.DEVICE = args.device
cfg.MODEL.RPN.NMS_THRESH = 0.7
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.4
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.DATASETS.TRAIN = ("mydataset_train",)
cfg.DATASETS.TEST = ("mydataset_val",)


cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.MAX_ITER = 300
cfg.SOLVER.STEPS = []        # do not decay learning rate
# faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
# only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = args.train_url
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
if args.device != 'cpu':
    trainer.train()


[12/12 01:59:51 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[12/12 01:59:55 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/12 02:00:23 d2.utils.events]:  eta: 0:06:36  iter: 19  total_loss: 4.152  loss_cls: 1.459  loss_box_reg: 0.2064  loss_mask: 0.6895  loss_rpn_cls: 1.64  loss_rpn_loc: 0.1084  time: 1.3737  data_time: 0.0511  lr: 1.6068e-05  max_mem: 2368M
[12/12 02:00:50 d2.utils.events]:  eta: 0:06:00  iter: 39  total_loss: 2.431  loss_cls: 1.008  loss_box_reg: 0.2761  loss_mask: 0.6784  loss_rpn_cls: 0.2617  loss_rpn_loc: 0.1421  time: 1.3560  data_time: 0.0129  lr: 3.2718e-05  max_mem: 2552M
[12/12 02:01:16 d2.utils.events]:  eta: 0:05:26  iter: 59  total_loss: 1.733  loss_cls: 0.6093  loss_box_reg: 0.2615  loss_mask: 0.6525  loss_rpn_cls: 0.05065  loss_rpn_loc: 0.171  time: 1.3446  data_time: 0.0105  lr: 4.9367e-05  max_mem: 2552M
[12/12 02:01:43 d2.utils.events]:  eta: 0:04:59  iter: 79  total_loss: 1.531  loss_cls: 0.3502  loss_box_reg: 0.3046  loss_mask: 0.6207  loss_rpn_cls: 0.05356  loss_rpn_loc: 0.1474  time: 1.3427  data_time: 0.0115  lr: 6.6017e-05  max_mem: 2552M
[12/12 02:02:11 d2.util

In [7]:
# 定义模型并训练

# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.MODEL.WEIGHTS = args.train_url+"/model_final.pth"

# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
# path to the model we just trained
evaluator = COCOEvaluator("mydataset_val", output_dir=cfg.OUTPUT_DIR)

val_loader = build_detection_test_loader(cfg, "mydataset_val")
predictor = DefaultPredictor(cfg)
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`
#model = trainer.build_model(cfg)
#metrics = trainer.test(cfg, model, evaluator)

demo = VisualizationDemo(cfg)
# 从文件中获取要拷贝的文件的信息


[12/12 02:06:50 d2.data.datasets.coco]: Loaded 6 images in COCO format from detectron2electric/dataset/coco-20211212/annotations/instances_val2014.json
[12/12 02:06:50 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/12 02:06:50 d2.data.common]: Serializing 6 elements to byte tensors and concatenating them all ...
[12/12 02:06:50 d2.data.common]: Serialized dataset takes 0.01 MiB
[12/12 02:06:51 d2.evaluation.evaluator]: Start inference on 6 batches


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


[12/12 02:06:57 d2.evaluation.evaluator]: Total inference time: 0:00:00.917471 (0.917471 s / iter per device, on 1 devices)
[12/12 02:06:57 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.376852 s / iter per device, on 1 devices)
[12/12 02:06:57 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[12/12 02:06:57 d2.evaluation.coco_evaluation]: Saving results to model/glue_hole/coco_instances_results.json
[12/12 02:06:57 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[12/12 02:06:57 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[12/12 02:06:57 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[12/12 02:06:57 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[12/12 02:06:57 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.00 seconds.
 Average Precisio

In [8]:
import os
filePath = os.path.join(args.data_url,args.dataset,"test2014")
list_data=os.listdir(filePath)
for filename in list_data:
  im = cv2.imread(os.path.join(filePath, filename))
  predictions, visualized_output = demo.run_on_image(im)
  visualized_output.save(os.path.join(args.train_url, filename))

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


In [12]:
import os
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
filePath = os.path.join("detectron2electric/JPEGImages")
list_data=os.listdir(filePath)
for filename in list_data:
  im = cv2.imread(os.path.join(filePath, filename))
  predictions, visualized_output = demo.run_on_image(im)
  visualized_output.save(os.path.join(args.train_url, filename))

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
